In [1]:
import numpy as np
import pandas as pd
import re
import nltk
df = pd.read_csv("AI_CRJ700.csv",header=0)
df1 = pd.read_csv("FI_CRJ700.csv",header=0)
df1 = df1[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df = df[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df2 = df.append(df1, ignore_index=True)


D:\Programs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df2['features'] = df2['INTERRUPTION_REASON'].str.cat(df2['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')
df2.head()

,ATA_SYMPTOM,ATA_CAUSE,INTERRUPTION_REASON,CORRECTIVE_ACTION,features
0,3081.0,308102,ICE DET (X) FAIL - STATUS,DEFERRED - REPLACED ICE DETECTOR,ICE DET (X) FAIL - STATUS DEFERRED - REPLACED...
1,2431.0,243101,TRU FAN FAIL - STATUS,DEFERRED - REPLACED TRU,TRU FAN FAIL - STATUS DEFERRED - REPLACED TRU
2,3613.0,361315,ISOL FAIL - CAUTION,REPLACED ISOLATION VALVE,ISOL FAIL - CAUTION REPLACED ISOLATION VALVE
3,3022.0,302201,Incident: RTRN L COWL ANTI ICE CAUTION MSG. L ...,(Resolved On W/O: 684248 Taskcard: NR-00001 ) ...,Incident: RTRN L COWL ANTI ICE CAUTION MSG. L ...
4,213108.0,213108,Incident: RTRN OVBD COOL CAUTION MESSAGE DISPL...,(Resolved On W/O: 684350 Taskcard: NR-00001 ) ...,Incident: RTRN OVBD COOL CAUTION MESSAGE DISPL...


In [3]:
len(df2)

394817

In [4]:
import re
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def identify_tokens1(df):
    comment = df['features']
    comment = re.sub(r'\W',' ',comment)
    comment = re.sub(r'\s+',' ', comment)
    #words = nltk.word_tokenize(comment)
   # new_words = [stemmer.stem(word) for word in words]
    #comment = ' '.join(new_words)
    return comment

df2['features_1'] = df2.apply(identify_tokens1, axis=1)

In [6]:
df_2020 = pd.read_csv("AI_2020.csv",header=0)
df_2020_1 = pd.read_csv("FI_2020.csv",header=0)
df_2020 = df_2020[df_2020['AC_MODEL']=='CRJ700']
df_2020_1 = df_2020_1[df_2020_1['AC_MODEL']=='CRJ700']
df_2020 = df_2020[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df_2020_1 = df_2020_1[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df_2020_2 = df_2020.append(df_2020_1, ignore_index=True)
len(df_2020_2)

3035

In [7]:
df_2020_2.head()

,ATA_SYMPTOM,ATA_CAUSE,INTERRUPTION_REASON,CORRECTIVE_ACTION
0,0,0,Reason: DUMMY RECORD,NaN
1,0,0,Reason: DUMMY RECORD,NaN
2,276201,276201,FLT SPLRS/SPOILERONS - CAUTION,REPLACED MFS PCU
3,5270,5211,PAX DR LATCH - CAUTION,DEFERRED - REPLACED LEVER ASSY
4,200,200,PAPERWORK - LOGBOOK,CORRECTED


In [8]:
#df1 = df.dropna(axis=0, subset=['ATA_SYMPTOM'])
df_2020_2['features'] = df_2020_2['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')
df_2020_2['feature_1'] = df_2020_2.apply(identify_tokens1, axis=1)


In [9]:
df_2020_2.head()

,ATA_SYMPTOM,ATA_CAUSE,INTERRUPTION_REASON,CORRECTIVE_ACTION,features,feature_1
0,0,0,Reason: DUMMY RECORD,NaN,Reason: DUMMY RECORD DEFERRED - REPLACED ICE ...,Reason DUMMY RECORD DEFERRED REPLACED ICE DETE...
1,0,0,Reason: DUMMY RECORD,NaN,Reason: DUMMY RECORD DEFERRED - REPLACED TRU,Reason DUMMY RECORD DEFERRED REPLACED TRU
2,276201,276201,FLT SPLRS/SPOILERONS - CAUTION,REPLACED MFS PCU,FLT SPLRS/SPOILERONS - CAUTION REPLACED ISOLA...,FLT SPLRS SPOILERONS CAUTION REPLACED ISOLATIO...
3,5270,5211,PAX DR LATCH - CAUTION,DEFERRED - REPLACED LEVER ASSY,PAX DR LATCH - CAUTION (Resolved On W/O: 6842...,PAX DR LATCH CAUTION Resolved On W O 684248 Ta...
4,200,200,PAPERWORK - LOGBOOK,CORRECTED,PAPERWORK - LOGBOOK (Resolved On W/O: 684350 ...,PAPERWORK LOGBOOK Resolved On W O 684350 Taskc...


CREATE df,df_2020 and RUN ATA_CAUSE

In [10]:
from sklearn.model_selection import train_test_split
X_train = df2['features_1']
y_train = df2['ATA_CAUSE']
X_test = df_2020_2['feature_1']
y_test= df_2020_2['ATA_CAUSE']

In [11]:
len(X_train)

394817

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer(max_features=4500,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=4500, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [21]:
X_train[13]

'Reason DUMMY RECORD '

In [18]:
X_test

0       Reason DUMMY RECORD DEFERRED REPLACED ICE DETE...
1               Reason DUMMY RECORD DEFERRED REPLACED TRU
2       FLT SPLRS SPOILERONS CAUTION REPLACED ISOLATIO...
3       PAX DR LATCH CAUTION Resolved On W O 684248 Ta...
4       PAPERWORK LOGBOOK Resolved On W O 684350 Taskc...
                              ...                        
3030    Left drain fault light on forward potable wate...
3031                            MDC FAULT status message 
3032    D FORWARD LAV SINK RUNS CONTINUOSLY C MEL 38 1...
3033    FA reports all seat outlets do not work CB in ...
3034    Deep hole about the size of a nickel found in ...
Name: feature_1, Length: 3035, dtype: object

In [22]:
from sklearn import metrics
predictions = text_clf.predict(X_test)
metrics.accuracy_score(y_test,predictions)

0.27479406919275123

In [24]:
output=pd.DataFrame(data={"Real":y_test,"prediction":predictions,"Features_Preprocessed":X_test}) 
#output
output.to_csv("Prediction_AI_ATA_SYM_CR700_new_dataset.csv",index=False,quoting=3,sep=',',escapechar='\\')

ATA_SYM

In [28]:
df1 = df.dropna(axis=0, subset=['ATA_SYMPTOM'])
df1_2020 = df_2020.dropna(axis=0, subset=['ATA_SYMPTOM'])
X_train = df1['features_1']
y_train = df1['ATA_SYMPTOM']
X_test = df1_2020['feature_1']
y_test= df1_2020['ATA_SYMPTOM']

In [29]:
text_clf = Pipeline([('tfidf',TfidfVectorizer(max_features=9000,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=9000, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [30]:
predictions = text_clf.predict(X_test)
metrics.accuracy_score(y_test,predictions)

0.59375